In [2]:
import sys; sys.path.append('..')
from common.postgresql import PostgresConnector as db
import os
import pandas as pd
import re
from sqlalchemy import text

# Configurações
DEST_DIR = r"E:/Download/cvm"
FULL_CLEAN_INSTALL = True  # Definido como True para sua reinstalação

connector = db()
tables_cleaned = set()

def fast_bulk_ingest(df, table_name, file_name, is_first_file):
    """
    Função de alta performance para o Clean Ingest.
    Se for o primeiro arquivo e FULL_CLEAN_INSTALL estiver ativo, dá DROP.
    Senão, apenas faz o append direto (sem WHERE) para máxima velocidade.
    """
    s_quoted, t_quoted, s_raw, t_raw = connector._split_table(table_name)
    
    with connector.engine.begin() as conn:
        # 1. Lógica de Drop/Create (Apenas uma vez por tabela)
        if is_first_file:
            conn.execute(text(f"DROP TABLE IF EXISTS {s_quoted}.{t_quoted} CASCADE;"))
            # Cria a estrutura e o __id (seu padrão salvo)
            df.head(0).to_sql(t_raw, conn, schema=s_raw, if_exists='replace', index=False)
            conn.execute(text(f"ALTER TABLE {s_quoted}.{t_quoted} ADD COLUMN __id SERIAL PRIMARY KEY;"))
        
        # 2. Ingestão em Massa (Append direto)
        # O 'method=multi' agrupa múltiplos inserts em uma única query, acelerando muito
        df.to_sql(t_raw, conn, schema=s_raw, if_exists='append', index=False, method='multi', chunksize=10000)

def clean_table_name(relative_dir, file_name):
    """Aplica as regras de limpeza de nome de tabela."""
    path_parts = relative_dir.replace('\\', '/').split('/')
    if 'DADOS' in path_parts: path_parts.remove('DADOS')
    if 'META' in path_parts: return None
    
    schema_prefix = "cvm." + "_".join(path_parts).lower()
    name_base = os.path.splitext(file_name)[0]
    name_base = re.sub(r'_?\d{4,6}', '', name_base).strip('_').lower()

    full_name = f"{schema_prefix}_{name_base}"
    parts = full_name.split('_')
    clean_parts = []
    for i, p in enumerate(parts):
        if i == 0 or p != parts[i-1]:
            clean_parts.append(p)
    return "_".join(clean_parts)

def process_local_files():
    print(f"Iniciando Ingestão Local Massiva...")
    
    for root, dirs, files in os.walk(DEST_DIR):
        if 'META' in root.upper(): continue

        for file in files:
            if file.lower().endswith('.csv'):
                file_path = os.path.join(root, file)
                relative_dir = os.path.relpath(root, DEST_DIR)
                table_name = clean_table_name(relative_dir, file) # Sua função de nome
                
                if not table_name: continue

                try:
                    # Carregamento rápido: selecionamos colunas e definimos tipos se possível
                    df = pd.read_csv(file_path, sep=';', encoding='iso-8859-1', low_memory=False)
                    df.columns = [c.lower() for c in df.columns]
                    df['__file'] = file # Metadado solicitado
                    
                    # Verifica se é a primeira vez que mexemos nesta tabela nesta sessão
                    is_first = False
                    if table_name not in tables_cleaned:
                        if FULL_CLEAN_INSTALL:
                            is_first = True
                        tables_cleaned.add(table_name)

                    print(f"Ingerindo: {file} -> {table_name}")
                    fast_bulk_ingest(df, table_name, file, is_first)
                    
                except Exception as e:
                    print(f"Erro em {file}: {e}")

# (Mantenha a função clean_table_name igual à anterior)
process_local_files()

KeyboardInterrupt: 